In [2]:
import os
local_finder_addition = "/Users/anna/sftp_mount/hpc_disk/02-awegmann/"

In [61]:
path_gpt2_PAN = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/mixed-gpt2-32000/749M/steps-45000/seed-42/42/PAN")
path_llama3_PAN = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/mixed-llama3-32000/749M/steps-45000/seed-42/42/PAN")
path_wsorg_PAN = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/mixed-wsorg-32000/749M/steps-45000/seed-42/42/PAN")
path_ws_PAN = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/mixed-ws-32000/749M/steps-45000/seed-42/42/PAN")
tokenizer_gpt2 = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/mixed-gpt2-32000")
tokenizer_llama3 = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/mixed-llama3-32000")
tokenizer_wsorg = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/mixed-wsorg-32000")
tokenizer_ws = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/mixed-ws-32000")
path_gpt2_webbook_PAN = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/base-BERT/mixed-gpt2-32000/749M/steps-45000/seed-42/42/PAN")
path_llama3_webbook_PAN = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/base-BERT/mixed-llama3-32000/749M/steps-45000/seed-42/42/PAN")

In [62]:
# laod tokeinzer
from train_bert import load_tokenizer
gpt2_tok, _ = load_tokenizer(tokenizer_gpt2)
llama3_tok, _ = load_tokenizer(tokenizer_llama3)
wsorg_tok, _ = load_tokenizer(tokenizer_wsorg)
ws_tok, _ = load_tokenizer(tokenizer_ws)

2025-02-06 17:34:38,407 - styletokenizer.utility.custom_logger - DEBUG - Loading previously fitted tokenizer from local path: /Users/anna/sftp_mount/hpc_disk/02-awegmann/TOKENIZER/tokenizer/mixed-gpt2-32000/tokenizer.json
2025-02-06 17:34:38,865 - styletokenizer.utility.custom_logger - DEBUG - Loading previously fitted tokenizer from local path: /Users/anna/sftp_mount/hpc_disk/02-awegmann/TOKENIZER/tokenizer/mixed-llama3-32000/tokenizer.json
2025-02-06 17:34:39,291 - styletokenizer.utility.custom_logger - DEBUG - Loading previously fitted tokenizer from local path: /Users/anna/sftp_mount/hpc_disk/02-awegmann/TOKENIZER/tokenizer/mixed-wsorg-32000/tokenizer.json
2025-02-06 17:34:39,754 - styletokenizer.utility.custom_logger - DEBUG - Loading previously fitted tokenizer from local path: /Users/anna/sftp_mount/hpc_disk/02-awegmann/TOKENIZER/tokenizer/mixed-ws-32000/tokenizer.json


In [43]:
import pandas as pd
df_gpt2 = pd.read_csv(os.path.join(path_gpt2_PAN, "2025-01-21_eval_dataset.tsv"), sep="\t")
df_wb_gpt2 = pd.read_csv(os.path.join(path_gpt2_webbook_PAN, "2025-01-10_eval_dataset.tsv"), sep="\t")
df_llama3 = pd.read_csv(os.path.join(path_llama3_PAN, "2025-01-22_eval_dataset.tsv"), sep="\t")
df_wb_llama3 = pd.read_csv(os.path.join(path_llama3_webbook_PAN, "2025-01-10_eval_dataset.tsv"), sep="\t")

In [53]:
df_wsorg = pd.read_csv(os.path.join(path_wsorg_PAN, "2025-01-25_eval_dataset.tsv"), sep="\t")

In [63]:
df_ws = pd.read_csv(os.path.join(path_ws_PAN, "2025-01-26_eval_dataset.tsv"), sep="\t")

In [64]:
# find cases where llama3 predicted correctly, but gpt2 did not
df_llama3_correct_gpt2_wrong = df_llama3[(df_llama3["label"] == df_llama3["predictions"]) & (df_gpt2["label"] != df_gpt2["predictions"])]
df_wb_gpt2_correct_llama3_wrong = df_wb_gpt2[(df_wb_gpt2["label"] == df_wb_gpt2["predictions"]) & (df_llama3["label"] != df_llama3["predictions"])]
df_ws_correct_gpt2_wrong = df_ws[(df_ws["label"] == df_ws["predictions"]) & (df_gpt2["label"] != df_gpt2["predictions"])]

In [65]:
print(len(df_llama3_correct_gpt2_wrong))
print(len(df_wb_gpt2_correct_llama3_wrong))
print(len(df_ws_correct_gpt2_wrong))

531
434
447


In [66]:
# note that 0 for PAN corresponds to same author or no change, see https://pan.webis.de/clef24/pan24-web/style-change-detection.html
i = 0
first_tok = gpt2_tok
first_name = "GPT2"
second_tok = llama3_tok
second_name = "LLAMA3"
second_tok = ws_tok
second_name = "ws"
for index, row in df_ws_correct_gpt2_wrong.iterrows():
    print("==Same Author==" if row["predictions"] == 0 else "==Different Author==")
    print(f"Text 1: {row['text 1'][:300]}")
    print(f"\t Tokenized {first_name}:{' '.join(first_tok.tokenize(row['text 1'][:300]))}")
    print(f"\t Tokenized {second_name}:{' '.join(second_tok.tokenize(row['text 1'][:300]))}")
    print(f"\t Unique Tokens {first_name}: {set(first_tok.tokenize(row['text 1'])) - set(second_tok.tokenize(row['text 1']))}")
    print(f"\t Unique Tokens {second_name}: {set(second_tok.tokenize(row['text 1'])) - set(first_tok.tokenize(row['text 1']))}")
    print(f"Text 2: {row['text 2'][:300]}")   
    print(f"\t Tokenized {first_name}:{' '.join(first_tok.tokenize(row['text 2'][:300]))}")
    print(f"\t Tokenized {second_name}:{' '.join(second_tok.tokenize(row['text 2'][:300]))}")
    print(f"\t Unique Tokens {first_name}: {set(first_tok.tokenize(row['text 2'])) - set(second_tok.tokenize(row['text 2']))}")
    print(f"\t Unique Tokens {second_name}: {set(second_tok.tokenize(row['text 2'])) - set(first_tok.tokenize(row['text 2']))}")
    i += 1
    
    if i > 10: break

==Different Author==
Text 1: Yeah but use your brain for a second and look into what created the circumstances for those genocides or mass executions to even be possible. Killing 15,000 people is a big ask, who is going to dish it out? If you are using soldiers and bullets you have the nazi problem: How do you handle the moral 
	 Tokenized GPT2:Yeah Ġbut Ġuse Ġyour Ġbrain Ġfor Ġa Ġsecond Ġand Ġlook Ġinto Ġwhat Ġcreated Ġthe Ġcircumstances Ġfor Ġthose Ġgen oc ides Ġor Ġmass Ġexecut ions Ġto Ġeven Ġbe Ġpossible . ĠK illing Ġ15 , 000 Ġpeople Ġis Ġa Ġbig Ġask , Ġwho Ġis Ġgoing Ġto Ġdish Ġit Ġout ? ĠIf Ġyou Ġare Ġusing Ġsoldiers Ġand Ġbullets Ġyou Ġhave Ġthe Ġn azi Ġproblem : ĠHow Ġdo Ġyou Ġhandle Ġthe Ġmoral Ġ
	 Tokenized ws:Yeah Ġbut Ġuse Ġyour Ġbrain Ġfor Ġa Ġsecond Ġand Ġlook Ġinto Ġwhat Ġcreated Ġthe Ġcircumstances Ġfor Ġthose Ġgen oc ides Ġor Ġmass Ġexecut ions Ġto Ġeven Ġbe Ġpossible. ĠK illing Ġ15 ,000 Ġpeople Ġis Ġa Ġbig Ġask, Ġwho Ġis Ġgoing Ġto Ġdish Ġit Ġout? ĠIf Ġyou Ġare Ġusin

In [67]:
# load webbook corpus
path_webbook = os.path.join(local_finder_addition,
                             "TOKENIZER/data/train-corpora/webbook")

In [68]:
from datasets import load_from_disk
webbook = load_from_disk(path_webbook)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading dataset from disk:   0%|          | 0/40 [00:00<?, ?it/s]

In [70]:
# for the dev set print the first 10 texts
for i in range(10):
    print(f"Excerpt {i}: {webbook['train'][i]['text'][:300]}")

Excerpt 0:  visit and they all swore on a second blood oath that it wasn't them.

"What about Phantom?" Hell Girl asked. "Maybe Phantom talked to Campus News."

Frankenstein shook his head. "Why? Skeletor has pictures of Phantom like he has of the rest of us."

Ghost Face looked thoughtful. "Maybe Phantom want
Excerpt 1: The future of SA-affiliated club sports, a cappella, and Greek groups is uncertain after the All-Campus Judicial Council ruled Friday that they may not use gendered language in their constitutions, advertisements, and names, and that they may not participate in gender-exclusive competitions.

The ru
Excerpt 2:  to support it; then a chunk of Iceland spar was discovered among navigational instruments on an Elizabethan ship that sank in 1592. This wasn't a Viking ship, but it existed almost two hundred years before scientists even understood the concept of using a crystal to determine direction at dawn and 
Excerpt 3:  in his Trojan maiden. Statius's Antigone, like all t

In [71]:
path_gpt2_mnli = os.path.join(local_finder_addition, "TOKENIZER/output/GLUE/train-mixed/base-BERT/mixed-gpt2-32000/749M/steps-45000/seed-42/42/mnli")
path_llama3_mnli = os.path.join(local_finder_addition, "TOKENIZER/output/GLUE/train-mixed/base-BERT/mixed-llama3-32000/749M/steps-45000/seed-42/42/mnli")
df_mnli_gpt2 = pd.read_csv(os.path.join(path_gpt2_mnli, "2025-02-01_eval_dataset_mnli.tsv"), sep="\t")
df_mnli_llama3 = pd.read_csv(os.path.join(path_llama3_mnli, "2025-01-22_eval_dataset_mnli.tsv"), sep="\t")

In [72]:
df_mnli_gpt2_correct_llama3_wrong = df_mnli_gpt2[(df_mnli_gpt2["label"] == df_mnli_gpt2["predictions"]) & (df_mnli_llama3["label"] != df_mnli_llama3["predictions"])]

In [73]:
print(len(df_mnli_gpt2_correct_llama3_wrong))

619


In [79]:
i = 0
first_tok = gpt2_tok
first_name = "GPT2"
second_tok = llama3_tok
second_name = "LLAMA3"
for index, row in df_mnli_gpt2_correct_llama3_wrong.iterrows():
    if row["predictions"] == 0:
        print("==entailment==")
    elif row["predictions"] == 1:
        print("==neutral==")
    elif row["predictions"] == 2:
        print("==contradiction==")
    print(f"Text 1: {row['premise'][:300]}")
    print(f"\t Tokenized {first_name}:{' '.join(first_tok.tokenize(row['premise'][:300]))}")
    print(f"\t Tokenized {second_name}:{' '.join(second_tok.tokenize(row['premise'][:300]))}")
    print(f"\t Unique Tokens {first_name}: {set(first_tok.tokenize(row['premise'])) - set(second_tok.tokenize(row['premise']))}")
    print(f"\t Unique Tokens {second_name}: {set(second_tok.tokenize(row['premise'])) - set(first_tok.tokenize(row['premise']))}")
    print(f"Text 2: {row['hypothesis'][:300]}")   
    print(f"\t Tokenized {first_name}:{' '.join(first_tok.tokenize(row['hypothesis'][:300]))}")
    print(f"\t Tokenized {second_name}:{' '.join(second_tok.tokenize(row['hypothesis'][:300]))}")
    print(f"\t Unique Tokens {first_name}: {set(first_tok.tokenize(row['hypothesis'])) - set(second_tok.tokenize(row['hypothesis']))}")
    print(f"\t Unique Tokens {second_name}: {set(second_tok.tokenize(row['hypothesis'])) - set(first_tok.tokenize(row['hypothesis']))}")
    i += 1
    
    if 20 > i > 50: break

==entailment==
Text 1: uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him
	 Tokenized GPT2:uh Ġi Ġdon 't Ġknow Ġi Ġi Ġhave Ġmixed Ġemotions Ġabout Ġhim Ġuh Ġsometimes Ġi Ġlike Ġhim Ġbut Ġat Ġthe Ġsame Ġtimes Ġi Ġlove Ġto Ġsee Ġsomebody Ġbeat Ġhim
	 Tokenized LLAMA3:uh Ġi Ġdon 't Ġknow Ġi Ġi Ġhave Ġmixed Ġemotions Ġabout Ġhim Ġuh Ġsometimes Ġi Ġlike Ġhim Ġbut Ġat Ġthe Ġsame Ġtimes Ġi Ġlove Ġto Ġsee Ġsomebody Ġbeat Ġhim
	 Unique Tokens GPT2: set()
	 Unique Tokens LLAMA3: set()
Text 2: I like him for the most part, but would still enjoy seeing someone beat him.
	 Tokenized GPT2:I Ġlike Ġhim Ġfor Ġthe Ġmost Ġpart , Ġbut Ġwould Ġstill Ġenjoy Ġseeing Ġsomeone Ġbeat Ġhim .
	 Tokenized LLAMA3:I Ġlike Ġhim Ġfor Ġthe Ġmost Ġpart , Ġbut Ġwould Ġstill Ġenjoy Ġseeing Ġsomeone Ġbeat Ġhim .
	 Unique Tokens GPT2: set()
	 Unique Tokens LLAMA3: set()
==contradiction==
Text 1: yeah i i think my favorite restaurant is always been